In [1]:
import sys
sys.path
sys.path.append('../../L1DeepMETv2/')
from graphmetnetwork import GraphMetNetwork

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.utils import to_undirected
from torch_cluster import radius_graph, knn_graph
from torch_geometric.datasets import MNISTSuperpixels
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from tqdm import tqdm
import model.net as net
import model.data_loader as data_loader

/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load Parameters

In [2]:
weights = '../weights/'
data_dir = '../../L1DeepMETv2/data_ttbar'

In [3]:
dataloaders = data_loader.fetch_dataloader(data_dir = data_dir, batch_size=6, validation_split=.2)
train_dl = dataloaders['train']
test_dl = dataloaders['test']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Training dataloader: {}, Test dataloader: {}'.format(len(train_dl), len(test_dl)))
print(device)

Processing...
0it [00:00, ?it/s]

Split: 199708
length of train/val data:  798834 199708
Training dataloader: 133139, Test dataloader: 33285
cuda



Done!
/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [4]:
test_data = None
for cnt, test_data in enumerate(test_dl):
    if cnt == 5:
        break

#### Load Tensor Parameters

In [5]:
n_features_cont = 6
x_cont_test = test_data.x[:,:n_features_cont] .to(device)  # include puppi
x_cat_test = test_data.x[:,n_features_cont:].long().to(device)
etaphi_test = torch.cat([test_data.x[:, 3][:, None], test_data.x[:, 4][:, None]], dim=1).to(device=device)
batch_test = test_data.batch.to(device)
edge_index_test = radius_graph(etaphi_test, r=0.4, batch=batch_test, loop=False, max_num_neighbors=255).to(device=device)
print(f'x_cont_test: {x_cont_test.shape}')
print(f'x_cat_test: {x_cat_test.shape}')
print(f'etaphi: {etaphi_test.shape}')
print(f'batch: {batch_test.shape}')
print(f'edge_index: {edge_index_test.shape}')

/nethome/dvsm3/.conda/envs/deepmet/lib/python3.10/site-packages/torch/cuda/__init__.py:146: UserWarning: 
NVIDIA RTX A6000 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA RTX A6000 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


x_cont_test: torch.Size([267, 6])
x_cat_test: torch.Size([267, 2])
etaphi: torch.Size([267, 2])
batch: torch.Size([267])
edge_index: torch.Size([2, 1494])


#### Convert Tensor parameters to Numpy arrays

In [7]:
x_cont = np.ascontiguousarray(x_cont_test.squeeze(0).cpu().numpy())
x_cat = np.ascontiguousarray(x_cat_test.squeeze(0).cpu().numpy())
batch = np.ascontiguousarray(batch_test.squeeze(0).cpu().numpy())
etaphi = etaphi_test.squeeze(0).cpu().numpy()
edge_index = edge_index_test.squeeze(0).cpu().numpy().transpose()
num_nodes = x_cont.shape[0]
batch_size = batch.shape[0]
print(f'Number of nodes: {num_nodes}')
assert(num_nodes == batch_size)

Number of nodes: 267


### Load the C++ Model

In [8]:
# Create an instance of the C++ GraphMetNetwork model
cmodel = GraphMetNetwork()

# Load the weights
cmodel.load_weights(weights)

### Run the Model

In [9]:
cmodel.GraphMetNetworkLayers(x_cont, x_cat, batch, num_nodes)

#### Test Inputs

In [20]:
c_x_cont = cmodel.get_x_cont()
c_x_cat = cmodel.get_x_cat()
c_batch = cmodel.get_batch()
c_num_nodes = cmodel.get_num_nodes()
print(f'Shape of c_x_cont: {c_x_cont.shape}')
print(f'Shape of c_x_cat: {c_x_cat.shape}')
print(f'Shape of c_batch: {c_batch.shape}')
print(f'Value of c_num_nodes: {c_num_nodes}')

Shape of c_x_cont: (267, 6)
Shape of c_x_cat: (267, 2)
Shape of c_batch: (267,)
Value of c_num_nodes: 267


In [18]:
# print(x_cont)
assert(np.allclose(x_cont, c_x_cont, atol=1e-5))
assert(np.allclose(x_cat, c_x_cat, atol=1e-5))
assert(np.allclose(batch, c_batch, atol=1e-5))
assert(np.allclose(num_nodes, c_num_nodes, atol=1e-5))



#### Test Internal Variables

In [26]:
c_etaphi = cmodel.get_etaphi()
print(etaphi.shape)
print(c_etaphi.shape)
print(type(etaphi))
print(type(c_etaphi))
are_almost_equal = np.allclose(etaphi, c_etaphi, atol=1e-5)
assert(np.allclose(etaphi, c_etaphi, atol=1e-5))

(267, 2)
(267, 2)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [31]:
c_num_edges = cmodel.get_num_edges()
c_edge_index = cmodel.get_edge_index()
edge_index_np = edge_index_test.squeeze(0).cpu().numpy().transpose()
print(edge_index_np.shape)
print(c_edge_index.shape)
print(f'Number of C edges: {c_num_edges}')

(1494, 2)
(1494, 2)
Number of C edges: 1494


In [ ]:
print(edge_index_np)
print(c_edge_index)

#### Test Intermediate Variables

### Other Tests

In [ ]:
import c_radius_graph

# Example points in 2D space and their corresponding batch indices
points = [[0.0, 0.0], [1.0, 1.0], [2.0, 2.0],  # Batch 0
          [3.0, 3.0], [4.0, 4.0],              # Batch 1
          [5.0, 5.0], [6.0, 6.0]]              # Batch 2
batch_indices = [0, 0, 0, 1, 1, 2, 2]  # Batch assignments
radius = 1.5

In [ ]:
# Call the C++ function
neighbors = c_radius_graph.find_neighbors_by_batch(points, batch_indices, radius)

# Print neighbors
for pair in neighbors:
    print(f"Point {pair[0]} is within radius of point {pair[1]}")

neighbors

In [ ]:
points_tensor = torch.tensor(points)
batch_tensor = torch.tensor(batch_indices)

edge_index_pts = radius_graph(points_tensor, r=radius, batch=batch_tensor, loop=False, max_num_neighbors=255)

In [ ]:
print(edge_index_pts)